In [1]:
import cv2

import numpy as np
import pandas as pd
import pickle

import os
import shutil

from ultralytics import YOLO

from tqdm import tqdm

from utils import init_metadata

In [2]:
metadata = init_metadata("./original-data/metadata_video.csv", "./original-data/metadata_imgs.csv")["metadata_img"]
metadata.head()

,width,height,color channels,path
title,,,,
snapshot cam29-01_12_23_12_01_22.png,720,1280,3,data/Артем Пушкина/snapshot cam29-01_12_23_12_...
snapshot cam67-15_11_23_18_01_37.png,720,1280,3,data/Артем Пушкина/snapshot cam67-15_11_23_18_...
snapshot cam11-15_11_23_09_59_08.png,720,1280,3,data/Артем Пушкина/snapshot cam11-15_11_23_09_...
snapshot cam13-29_11_23_19_38_30.png,720,1280,3,data/Артем Пушкина/snapshot cam13-29_11_23_19_...
snapshot cam11-17_11_23_10_03_46.png,720,1280,3,data/Артем Пушкина/snapshot cam11-17_11_23_10_...


In [3]:
paths = [path.replace("data", "original-data") for path in metadata["path"].values]
path_labels = [path.replace("original-data", "labels").replace(".png", ".pickle") for path in paths]

In [4]:
with open("./labels/path_labels_img.pickle", "wb") as f:
    pickle.dump(path_labels, f)

In [5]:
path_labels[:5]

['labels/Артем Пушкина/snapshot cam29-01_12_23_12_01_22.pickle',
 'labels/Артем Пушкина/snapshot cam67-15_11_23_18_01_37.pickle',
 'labels/Артем Пушкина/snapshot cam11-15_11_23_09_59_08.pickle',
 'labels/Артем Пушкина/snapshot cam13-29_11_23_19_38_30.pickle',
 'labels/Артем Пушкина/snapshot cam11-17_11_23_10_03_46.pickle']

In [6]:
paths[:5]

['original-data/Артем Пушкина/snapshot cam29-01_12_23_12_01_22.png',
 'original-data/Артем Пушкина/snapshot cam67-15_11_23_18_01_37.png',
 'original-data/Артем Пушкина/snapshot cam11-15_11_23_09_59_08.png',
 'original-data/Артем Пушкина/snapshot cam13-29_11_23_19_38_30.png',
 'original-data/Артем Пушкина/snapshot cam11-17_11_23_10_03_46.png']

In [7]:
model = YOLO("model/yolov8x.pt")

In [38]:
img = cv2.imread(paths[4])
pred = model(img, classes=0)
boxes = pred[0].boxes[0].xyxyn


0: 480x640 6 persons, 236.7ms
Speed: 4.8ms preprocess, 236.7ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)


In [50]:
boxes.tolist()

[0.5834288001060486,
 0.23490500450134277,
 0.6705202460289001,
 0.6010701656341553]

In [52]:
pred[0].names


{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [45]:
[int(i) for i in pred[0].boxes.cls.tolist()]

[0, 0, 0, 0, 0, 0]

In [14]:
def marking_img(
    model: YOLO,
    path: str,
):
    results = []
    img = cv2.imread(path)

    result = model.predict(img, classes=0, device="0", verbose=False)

    results.append(
        {
            "boxes": result[0].boxes,
            "probs": result[0].probs
        }
    )
    
    return pd.DataFrame(results)


In [16]:
for path, path_label in zip(tqdm(paths), path_labels):
    marks = marking_img(
        model=model,
        path=path
    )
    
    with open(path_label, "wb") as f:
        pickle.dump(marks, f)


  0%|          | 0/6629 [00:00<?, ?it/s]

100%|██████████| 6629/6629 [14:20<00:00,  7.71it/s]
